<a href="https://colab.research.google.com/github/laraselinseyahi/Deep-Learning/blob/main/CS230_Project_LaraSelinSeyahi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive/retino
%ls

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import os


# Data Pre-processing


In [ ]:
train_dir = 'train/'
val_dir = 'valid/'
test_dir = 'test/'

# Define image parameters
img_height = 224
img_width = 224
batch_size = 32

# Load training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Load validation dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Load test dataset
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False  # Important for evaluation; keep order consistent
)

In [ ]:
# helps keep memory in cache after it's been loaded from the disk
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
# normalization_layer = tf.keras.layers.Rescaling(1./255)
normalization_layer = tf.keras.applications.resnet.preprocess_input

In [ ]:
data_augmentation_layer = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

def data_augmentation_layer_function(inputs):
    return data_augmentation_layer(inputs)

# Apply the data augmentation to each element in the dataset
def apply_data_augmentation(image, label):
    # Apply augmentation only to the image (not the label)
    augmented_image = data_augmentation_layer(image)
    return augmented_image, label

# Define the normalization layer
def normalization_layer(inputs):
    return tf.keras.applications.resnet.preprocess_input(inputs)

# ResNet-50


In [ ]:
# RESNET

resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in resnet_model.layers:
  layer.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))  # Define the input shape
x = data_augmentation_layer(inputs)  # Apply data augmentation
x = normalization_layer(x)  # Apply normalization (preprocessing)
x = resnet_model(x)  # Pass through the base ResNet model
x = layers.GlobalAveragePooling2D()(x)  # Global average pooling layer
outputs = layers.Dense(1, activation='sigmoid')(x)  # Sigmoid for binary classification

# Create the model
model = models.Model(inputs, outputs)

base_learning_rate = 0.001

# since this is binary classification, loss is BCE
model.compile(optimizer=Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

# Train with frozen base model layers
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

In [ ]:
# RESNET Experimenting

resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in resnet_model.layers:
  layer.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))  # Define the input shape
x = data_augmentation_layer(inputs)  # Apply data augmentation
x = normalization_layer(x)  # Apply normalization (preprocessing)
x = resnet_model(x)  # Pass through the base ResNet model
x = layers.GlobalAveragePooling2D()(x)  # Global average pooling layer
x = layers.Dense(128, activation='relu')(x)  # Dense layer
x = layers.Dropout(0.5)(x)  # Dropout for regularization
outputs = layers.Dense(1, activation='sigmoid')(x)  # Sigmoid for binary classification # try with having on this dense layer! then add one by one

# Create the model
model = models.Model(inputs, outputs)

base_learning_rate = 0.001

# since this is binary classification, loss is BCE
model.compile(optimizer=Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

# Train with frozen base model layers
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

In [ ]:
# RESNET (Training 50 epochs and unfroze last 10 layers)

# delete top layer (include_top = false, make other layers non-trainable)
resnet_model_top = ResNet50(weights='imagenet', include_top=True, input_shape=(224, 224, 3))
for layer in resnet_model_top.layers[-2:]:
  print(layer)


# delete top layer (include_top = false, make other layers non-trainable)
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in resnet_model.layers[:-10]:
  layer.trainable = False

#x = normalization_layer(x)

# x = train_ds.map(apply_data_augmentation)
# x = normalization_layer(train_ds)

# on top of the base model, add layers for binary classification
# Build the model
inputs = tf.keras.Input(shape=(224, 224, 3))  # Define the input shape
x = data_augmentation_layer(inputs)  # Apply data augmentation
x = normalization_layer(x)  # Apply normalization (preprocessing)
x = resnet_model(x)  # Pass through the base ResNet model
x = layers.GlobalAveragePooling2D()(x)  # Global average pooling layer # TRY OUT different ways!! # hypothesis (ex: same performance can be achieved with less layers, reduce the number of layers)
x = layers.Dense(128, activation='relu')(x)  # Dense layer
outputs = layers.Dense(1, activation='sigmoid')(x)  # Sigmoid for binary classification # try with having on this dense layer! then add one by one

# Create the model
model = models.Model(inputs, outputs)

base_learning_rate = 0.001

# since this is binary classification, loss is BCE
model.compile(optimizer=Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

# Train with frozen base model layers
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50  # Start with a few epochs
)


# try not fine-tuning the model on the dataset below 94 (both accuracies)
# usually you have a hypothesis about the architecture

In [ ]:
import matplotlib.pyplot as plt

# Example data
x = [i for i in range(1, 51)]  # X-axis values
y = [0.8912, 0.9646, 0.9677, 0.9779, 0.9759, 0.9713, 0.9771, 0.9827, 0.9828, 0.9842, 0.9902, 0.9818, 0.9856, 0.9871, 0.9890, 0.9886, 0.9931, 0.9752, 0.9921, 0.9836, 0.9907, 0.9905, 0.9924, 0.9872, 0.9958, 0.9889, 0.9922, 0.9899, 0.9923, 0.9908, 0.9918, 0.9929, 0.9954, 0.9927, 0.9952, 0.9961, 0.9941, 0.9962, 0.9968, 0.9914, 0.9930, 0.9950, 0.9962, 0.9966, 0.9925, 0.9947, 0.9966, 0.9973, 0.9951, 0.9983]  # Y-axis values
print(len(y))
y = list(map(lambda x: x * 100, y))

plt.figure(figsize=(15, 5))  # Width = 15, Height = 5
val_acc = [0.9412, 0.9091, 0.9269, 0.8574, 0.9109, 0.9715, 0.9715, 0.9715, 0.9661, 0.9733, 0.9679, 0.9768, 0.9412, 0.9733, 0.9679, 0.9733, 0.9733, 0.9590, 0.9697, 0.9715, 0.9643, 0.9572, 0.9715, 0.9750, 0.9715, 0.9679, 0.9643, 0.9572, 0.8289, 0.9733, 0.9715, 0.9768, 0.9733, 0.9697, 0.9697, 0.9840, 0.9840, 0.9733, 0.9715, 0.9733, 0.9750, 0.9750, 0.9804, 0.9750, 0.9768, 0.9786, 0.9786, 0.9804, 0.9822, 0.9715]
val_acc = list(map(lambda x: x * 100, val_acc))

# Create the plot
plt.plot(x, y, label="Training Accuracy", color="blue", marker="o")
plt.plot(x, val_acc, label="Validation Accuracy", color="red", marker="o")

# Add labels and title
plt.xlabel("# of Epoch")
plt.ylabel("Accuracy %")
plt.title(" Accuracies Across Epochs")

# Add gridlines
plt.grid(True, linestyle='--', alpha=0.5)
plt.xticks(ticks=range(1, 51, 1))  # Show a tick every 2 numbers

# Add a legend
plt.legend()

# Show the plot
plt.show()


# AlexNet

In [ ]:
# ALEXNET Baseline Model
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets

from torch.utils.data import DataLoader

# Define transformations (resizing, normalization, etc.)
transform = transforms.Compose([
    transforms.Resize(256),       # Resize the image to 256x256
    transforms.CenterCrop(224),   # Crop the center of the image to 224x224
    transforms.ToTensor(),        # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for AlexNet
])

# Load dataset (for example, using ImageFolder)
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Load dataset (for example, using ImageFolder)
valid_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=True)

#should I instead write it from scratch with tensor flow?

# Load pre-trained AlexNet
alexnet = models.alexnet(pretrained=True)

# Freeze early layers (optional)
for param in alexnet.features.parameters():
    param.requires_grad = False

# Modify the classifier for binary classification
alexnet.classifier[6] = nn.Linear(alexnet.classifier[6].in_features, 2) # should I add av pooling, relu and sigmoid?

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet = alexnet.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(alexnet.parameters(), lr=0.001)

# Training loop (simplified)
num_epochs = 5
for epoch in range(num_epochs):
    alexnet.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = alexnet(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Print statistics for this epoch
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

    # Validation phase
    alexnet.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():  # Disable gradient computation for validation
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = alexnet(images)
            loss = criterion(outputs, labels)

            # Update validation metrics
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_loss /= len(valid_loader)
    val_accuracy = 100 * correct_val / total_val

    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")


# transformers


In [ ]:
# ALEXNET Experiments
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets

from torch.utils.data import DataLoader

# Define transformations (resizing, normalization, etc.)
transform = transforms.Compose([
    transforms.Resize(256),       # Resize the image to 256x256
    transforms.CenterCrop(224),   # Crop the center of the image to 224x224
    transforms.ToTensor(),        # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for AlexNet
])

# Load dataset (for example, using ImageFolder)
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Load dataset (for example, using ImageFolder)
valid_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=True)

#should I instead write it from scratch with tensor flow?

# Load pre-trained AlexNet
alexnet = models.alexnet(pretrained=True)

# Freeze early layers (optional)
for param in alexnet.features.parameters():
    param.requires_grad = False

# Modify the classifier
alexnet.classifier = nn.Sequential(
    *list(alexnet.classifier.children())[:-1],  # Remove the last layer
    nn.Linear(alexnet.classifier[6].in_features, 128),  # Add dense layer
    nn.ReLU(),
    nn.Linear(128, 2)  # Final output layer for 2 classes
)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet = alexnet.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(alexnet.parameters(), lr=0.001)

# Training loop (simplified)
num_epochs = 5
for epoch in range(num_epochs):
    alexnet.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = alexnet(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Print statistics for this epoch
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

    # Validation phase
    alexnet.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():  # Disable gradient computation for validation
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = alexnet(images)
            loss = criterion(outputs, labels)

            # Update validation metrics
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_loss /= len(valid_loader)
    val_accuracy = 100 * correct_val / total_val

    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

In [ ]:
# ALEXNET
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets

from torch.utils.data import DataLoader

# Define transformations (resizing, normalization, etc.)
transform = transforms.Compose([
    transforms.Resize(256),       # Resize the image to 256x256
    transforms.CenterCrop(224),   # Crop the center of the image to 224x224
    transforms.ToTensor(),        # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for AlexNet
])

# Load dataset (for example, using ImageFolder)
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Load dataset (for example, using ImageFolder)
valid_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=True)

#should I instead write it from scratch with tensor flow?

# Load pre-trained AlexNet
alexnet = models.alexnet(pretrained=True)

# Freeze early layers (optional)
for param in alexnet.features.parameters():
    param.requires_grad = False

# Modify the classifier
alexnet.classifier = nn.Sequential(
    *list(alexnet.classifier.children())[:-1],  # Remove the last layer
    nn.Linear(alexnet.classifier[6].in_features, 128),  # Add dense layer
    nn.ReLU(),  # ReLU activation
    nn.Linear(128, 128),  # Add dense layer
    nn.ReLU(),  # ReLU activation
    nn.Linear(128, 2)  # Final output layer for 2 classes
)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet = alexnet.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(alexnet.parameters(), lr=0.001)

# Training loop (simplified)
num_epochs = 5
for epoch in range(num_epochs):
    alexnet.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = alexnet(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Print statistics for this epoch
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

    # Validation phase
    alexnet.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():  # Disable gradient computation for validation
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = alexnet(images)
            loss = criterion(outputs, labels)

            # Update validation metrics
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_loss /= len(valid_loader)
    val_accuracy = 100 * correct_val / total_val

    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

In [ ]:
# ALEXNET
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets

from torch.utils.data import DataLoader

# Defining transformations (resizing, normalization, etc.)
transform = transforms.Compose([
    transforms.Resize(256),       # Resize the image to 256x256
    transforms.CenterCrop(224),   # Crop the center of the image to 224x224
    transforms.ToTensor(),        # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for AlexNet
])

# Load dataset (for example, using ImageFolder)
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Load dataset (for example, using ImageFolder)
valid_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=True)

# Load pre-trained AlexNet
alexnet = models.alexnet(pretrained=True)

# Freeze early layers (optional)
for param in alexnet.features.parameters():
    param.requires_grad = False

# Modify the classifier
alexnet.classifier = nn.Sequential(
    *list(alexnet.classifier.children())[:-1],  # Remove the last layer
    nn.Linear(alexnet.classifier[6].in_features, 128),  # Add dense layer
    nn.ReLU(),  # ReLU activation
    nn.Dropout(p=0.5),  # Dropout with a probability of 0.5
    nn.Linear(128, 2)  # Final output layer for 2 classes
)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet = alexnet.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(alexnet.parameters(), lr=0.001)

# Training loop (simplified)
num_epochs = 5
for epoch in range(num_epochs):
    alexnet.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = alexnet(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Print statistics for this epoch
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

    # Validation phase
    alexnet.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():  # Disable gradient computation for validation
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = alexnet(images)
            loss = criterion(outputs, labels)

            # Update validation metrics
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_loss /= len(valid_loader)
    val_accuracy = 100 * correct_val / total_val

    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

In [ ]:
# ALEXNET !!!
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets

from torch.utils.data import DataLoader

# Define transformations (resizing, normalization, etc.)
transform = transforms.Compose([
    transforms.Resize(256),       # Resize the image to 256x256
    transforms.CenterCrop(224),   # Crop the center of the image to 224x224
    transforms.ToTensor(),        # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for AlexNet
])

# Load dataset (for example, using ImageFolder)
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Load dataset (for example, using ImageFolder)
valid_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=True)

# Load pre-trained AlexNet
alexnet = models.alexnet(pretrained=True)

# Freeze early layers (optional)
for param in alexnet.features.parameters():
    param.requires_grad = False

# Modify the classifier
alexnet.classifier = nn.Sequential(
    *list(alexnet.classifier.children())[:-1],  # Remove the last layer
    nn.Linear(alexnet.classifier[6].in_features, 256),  # Add dense layer
    nn.ReLU(),  # ReLU activation
    nn.Linear(256, 2)  # Final output layer for 2 classes
)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet = alexnet.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(alexnet.parameters(), lr=0.001)

# Training loop (simplified)
num_epochs = 5
for epoch in range(num_epochs):
    alexnet.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = alexnet(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Print statistics for this epoch
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

    # Validation phase
    alexnet.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():  # Disable gradient computation for validation
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = alexnet(images)
            loss = criterion(outputs, labels)

            # Update validation metrics
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_loss /= len(valid_loader)
    val_accuracy = 100 * correct_val / total_val

    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

# InceptionNet

In [ ]:
# INCEPTIONNET

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
])

# Load InceptionNet without the top layer
inceptionnet_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

for layer in inceptionnet_model.layers:
  layer.trainable = False

model = models.Sequential([
    # Apply data augmentation to the input
    data_augmentation,
    # Resize the images to 299x299
    layers.Lambda(lambda x: tf.image.resize(x, (299, 299))),
    # Preprocess the image data using InceptionV3 preprocessing
    layers.Lambda(lambda x: tf.keras.applications.inception_v3.preprocess_input(x)), # normalization

    inceptionnet_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])


# train_ds = train_ds.map(lambda x, y: (tf.keras.applications.inception_v3.preprocess_input(data_augmentation(x)), y))

base_learning_rate = 0.001

# sincd this is binary classification, loss is BCE
model.compile(optimizer=Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

# Train with frozen base model layers
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5  # Start with a few epochs
)



In [ ]:
# INCEPTIONNET !!

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.2)
])

# Load InceptionNet without the top layer
inceptionnet_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

for layer in inceptionnet_model.layers:
  layer.trainable = False

model = models.Sequential([
    layers.Input(shape=(299, 299, 3)),  # Input can have dynamic size; adjust as needed

    # Apply data augmentation to the input
    data_augmentation,
    # Resize the images to 299x299
    layers.Lambda(lambda x: tf.image.resize(x, (299, 299))),
    # Preprocess the image data using InceptionV3 preprocessing
    layers.Lambda(lambda x: tf.keras.applications.inception_v3.preprocess_input(x)), # normalization

    inceptionnet_model,
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])


# train_ds = train_ds.map(lambda x, y: (tf.keras.applications.inception_v3.preprocess_input(data_augmentation(x)), y))

base_learning_rate = 0.001

# sincd this is binary classification, loss is BCE
model.compile(optimizer=Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

# Train with frozen base model layers
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5  # Start with a few epochs
)


In [ ]:
# INCEPTIONNET

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
])

# Load InceptionNet without the top layer
inceptionnet_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

for layer in inceptionnet_model.layers:
  layer.trainable = False

model = models.Sequential([
    layers.Input(shape=(299, 299, 3)),  # Input can have dynamic size; adjust as needed

    # Apply data augmentation to the input
    data_augmentation,
    # Resize the images to 299x299
    layers.Lambda(lambda x: tf.image.resize(x, (299, 299))),
    # Preprocess the image data using InceptionV3 preprocessing
    layers.Lambda(lambda x: tf.keras.applications.inception_v3.preprocess_input(x)), # normalization

    inceptionnet_model,
    layers.Dropout(0.5),
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

base_learning_rate = 0.001

# sincd this is binary classification, loss is BCE
model.compile(optimizer=Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

# Train with frozen base model layers
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5  # Start with a few epochs
)

In [ ]:
# INCEPTIONNET

# Load InceptionNet without the top layer
inceptionnet_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

for layer in inceptionnet_model.layers:
  layer.trainable = False

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
])

model = models.Sequential([
    layers.Input(shape=(299, 299, 3)),  # Input can have dynamic size; adjust as needed
    # Apply data augmentation to the input
    data_augmentation,    # Resize the images to 299x299
    layers.Lambda(lambda x: tf.image.resize(x, (299, 299))),
    # Preprocess the image data using InceptionV3 preprocessing
    layers.Lambda(lambda x: tf.keras.applications.inception_v3.preprocess_input(x)), # normalization
    inceptionnet_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

# train_ds = train_ds.map(lambda x, y: (tf.keras.applications.inception_v3.preprocess_input(data_augmentation(x)), y))

base_learning_rate = 0.001

# sincd this is binary classification, loss is BCE
model.compile(optimizer=Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

# Train with frozen base model layers
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)